In [1]:
import numpy as np
from Reader_functions import *

In [2]:
with open("Simulation0_SIM.txt", "r") as f:
    lines = f.readlines()

split_file = []
current_list = []

for line in lines:
    if line[0:3] == '***':
        split_file.append(current_list)
        current_list = []
    current_list.append(line.strip())
split_file.append(current_list)
split_file.pop(0)

for item in split_file:
    print(item)


['*** TIME INFORMATION ***', 'Total time: 1.00E-2', 'dt: 1.00E-8', 'dt_spring: 1.00E-9', 'dt_data: 1.00E-4', 'dt_image: 1.00E-4', '']
['*** SYSTEM INFORMATION ***', 'L_x: 0.1', 'L_y: 0.1', 'L_z_out: 0.01', 'L_z_in: 0.09', 'Partition Nx: 10', 'Partition Ny: 10', 'Partition Nz: 10', '']
['*** MOLECULES ***', '', 'MOLECULE: "R" Membrane Number 5 Site_Types 3 Total_Sites 4 Total_Links 4 is2D false', '{', 'TYPE: Name "R" Radius 2.00000 D 1.000 Color RED STATES "State0" "State1"', 'TYPE: Name "L" Radius 1.00000 D 1.000 Color ORANGE STATES "State0"', 'TYPE: Name "Anchor" Radius 1.00000 D 2.000 Color GRAY STATES "Anchor"', '', "SITE 0 : Intracellular : Initial State 'State0'", 'TYPE: Name "R" Radius 2.00000 D 1.000 Color RED STATES "State0" "State1"', 'x 0.00000 y 17.23333 z 50.15556', "SITE 1 : Membrane : Initial State 'Anchor'", 'TYPE: Name "Anchor" Radius 1.00000 D 2.000 Color GRAY STATES "Anchor"', 'x 0.00000 y 20.70000 z 37.00000', "SITE 2 : Intracellular : Initial State 'State0'", 'TYPE:

In [3]:
#Extract molecule data
molecules = []
molecule = []
for line in split_file[2]:
    if line[0:9] == 'MOLECULE:':
        molecules.append(molecule)
        molecule = []
    molecule.append(line)
molecules.append(molecule)    
molecules.pop(0)

for item in molecules:
    print(item)

['MOLECULE: "R" Membrane Number 5 Site_Types 3 Total_Sites 4 Total_Links 4 is2D false', '{', 'TYPE: Name "R" Radius 2.00000 D 1.000 Color RED STATES "State0" "State1"', 'TYPE: Name "L" Radius 1.00000 D 1.000 Color ORANGE STATES "State0"', 'TYPE: Name "Anchor" Radius 1.00000 D 2.000 Color GRAY STATES "Anchor"', '', "SITE 0 : Intracellular : Initial State 'State0'", 'TYPE: Name "R" Radius 2.00000 D 1.000 Color RED STATES "State0" "State1"', 'x 0.00000 y 17.23333 z 50.15556', "SITE 1 : Membrane : Initial State 'Anchor'", 'TYPE: Name "Anchor" Radius 1.00000 D 2.000 Color GRAY STATES "Anchor"', 'x 0.00000 y 20.70000 z 37.00000', "SITE 2 : Intracellular : Initial State 'State0'", 'TYPE: Name "R" Radius 2.00000 D 1.000 Color RED STATES "State0" "State1"', 'x 0.00000 y 24.41111 z 49.90556', "SITE 3 : Intracellular : Initial State 'State0'", 'TYPE: Name "L" Radius 1.00000 D 1.000 Color ORANGE STATES "State0"', 'x 0.00000 y 20.89444 z 42.11667', '', 'LINK: Site 2 ::: Site 0', 'LINK: Site 3 ::: S

In [61]:
#Molecule Processing
for item in molecules:   
    #General Info
    typStr = ''
    split_general_info = item[0].split()
    site_types = int(split_general_info[6])
    for j in range(2,2+site_types):
        type_split = item[j].split()
        type_name = type_split[2]
        states = [i for i in type_split if '"' in i]
        states.pop(0)
        typStrAddition = ''
        if len(states) == 1:
            pass
        else:
            for state in states:
                typStrAddition = typStrAddition + f'{state[1:-1]}, '
            typStrAddition = typStrAddition[:-2]
            typStrAddition = f' ({len(states)} possible states: {typStrAddition})'
        typStr = f'{typStr}{type_name[1:-1]}{typStrAddition}, '
    print(f'MOLECULE: {split_general_info[1]}\n')
    print(f'This molecule has {split_general_info[6]} site types: {typStr[:-2]}')
        
    #Sites
    total_sites = int(split_general_info[8])
    site_data = []
    for i in range(3 + site_types, 3 + site_types + 3*total_sites, 3):
        site_split1 = item[i].split()
        site_split2 = item[i+1].split()
        site = [f'{site_split1[0]} {site_split1[1]}', site_split1[3], site_split2[2][1:-1]]
        site_data.append(site)

    sites_reordered = []
    site_type_names = []  
    for site in site_data:
        if site[2] not in site_type_names:
            site_type_names.append(site[2])
            #Site type name, Numpy Array(#Intracellular, #Extracellular, #Membrane), [list of sites] 
            sites_reordered.append([site[2], location2vec(site[1]), [site[0]]])
        else:
            for reordered_site in sites_reordered:
                if site[2] == reordered_site[0]:
                    reordered_site[1] = reordered_site[1] + location2vec(site[1])
                    reordered_site[2].append(site[0])

    #Sites (default ordering)
    print(f'\nThis molecule consists of {total_sites} connected sites:')
    for site in site_data:
        print(f'{site[0]} ({site[1]}) of type "{site[2]}"')

    #Sites (grouped by type)
    print(f'\nThis molecule consists of {total_sites} connected sites:')
    for reordered_site in sites_reordered:
        str2 = 'Placeholder 2'
        location_list = list(reordered_site[1])
        site_list = list(reordered_site[2])
        for index in location_list:
            addStr = ''
            if location_list[0] != 0:
                addStr = addStr + f'{location_list[0]} intracellular, '
            if location_list[1] != 0:
                addStr = addStr + f'{location_list[1]} extracellular, '
            if location_list[2] != 0:
                addStr = addStr + f'{location_list[2]} membrane, '
        addStr2 = ''
        for index in site_list:
            addStr2 = addStr2 + index + ', '
        print(f'Type {reordered_site[0]}: {addStr[:-2]} sites ({addStr2[:-2]})')
    print('\n')

    total_links = int(split_general_info[10])
    link_data = []
    for i in range(4 + site_types + 3*total_sites, 4 + site_types + 3*total_sites + total_links):
        split_link = item[i].split()
        link = [split_link[2], split_link[5]]
        link_data.append(link)
    
    #[[link index, list of links],[link index, list of links],...]
    link_lists = []
    for site in site_data:
        link_count_init = [site[0][-1], []]
        link_lists.append(link_count_init)
    for link_list in link_lists:
        for link in link_data:
            if link_list[0] == link[0]:
                link_list[1].append(link[1])
            elif link_list[0] == link[1]:
                link_list[1].append(link[0])
            else:
                pass

    link_lists_remaining = link_lists

    while True:
        record_length = 0
        for link_list in link_lists:
            if len(link_list[1]) > record_length:
                record_length = len(link_list[1])
                record_loc = int(link_list[0])
        if record_length == 0:
            break
        site_str = ''
        for site in link_lists_remaining[record_loc][1]:
            site_str = f'{site_str} site {site},'
        
        print(f'Site {record_loc} is connected to{site_str[:-1]}')
        for link_list_remaining in link_lists_remaining:
            link_list_remaining[1] = [i for i in link_list_remaining[1] if i != str(record_loc)] 
        link_lists_remaining[record_loc][1] = []


MOLECULE: "R"

This molecule has 3 site types: R (2 possible states: State0, State1), L, Anchor

This molecule consists of 4 connected sites:
SITE 0 (Intracellular) of type "R"
SITE 1 (Membrane) of type "Anchor"
SITE 2 (Intracellular) of type "R"
SITE 3 (Intracellular) of type "L"

This molecule consists of 4 connected sites:
Type R: 2 intracellular sites (SITE 0, SITE 2)
Type Anchor: 1 membrane sites (SITE 1)
Type L: 1 intracellular sites (SITE 3)


Site 3 is connected to site 1, site 0, site 2
Site 0 is connected to site 2


MOLECULE: "Adapter"

This molecule has 2 site types: A, L

This molecule consists of 3 connected sites:
SITE 0 (Intracellular) of type "A"
SITE 1 (Intracellular) of type "L"
SITE 2 (Intracellular) of type "A"

This molecule consists of 3 connected sites:
Type A: 2 intracellular sites (SITE 0, SITE 2)
Type L: 1 intracellular sites (SITE 1)


Site 1 is connected to site 0, site 2


